In [10]:
#Скачиваем пакеты, которых нет в стандартной библиотеке Python
!pip install fake-useragent
!pip install tqdm

In [10]:
#Собственно выкачивание файлов из Интернета
import requests

#Работа с таблицами — понадобится, чтобы хранить наш корпус
import pandas as pd

# Модуль для работы с фаорматами времени
from datetime import datetime, timedelta

# Модуль для работы с HTML-файлами
from bs4 import BeautifulSoup

#Поможет замаскироваться под обычного пользователя в Интернете
from fake_useragent import UserAgent 

#Переменные, нужные, чтобы маскироваться под обычного пользователя Интернета
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

In [11]:
dates = []
n = 1
date_list = [datetime.today() - timedelta(days=x) for x in range(n)]
date_list = [f"{date.year}/{date.month}/{date.day}/" for date in date_list]

In [15]:
date_list

['2023/3/24/']

In [13]:
all_titles = []
all_dates = []
all_links = []
all_texts = []

for date in date_list:
    page = f"https://www.mk.ru/news/{date}"
    result = requests.get(page)
    html = result.text
    soup = BeautifulSoup(html)
    titles = soup.find_all('h3', {'class': "news-listing__item-title"})
    titles = [x.text.strip() for x in titles]
    all_titles.extend(titles)
    all_dates.extend([date]*len(titles))
    links = soup.find_all('a', {'class': "news-listing__item-link"}, href=True)
    links = [f"{link['href']}" for link in links]
    all_links.extend(links)
    for link in links:
        result = requests.get(link)
        html = result.text
        soup = BeautifulSoup(html)
        page_text = soup.find('div', {'itemprop': "articleBody"})
        lines = [line.text.strip() for line in page_text.find_all('p')]
        all_texts.extend([' '.join(lines).replace('\xa0', "").replace('\n', ' ')])

In [14]:
#Создаём индекс, чтобы пронумеровать каждый текст

index = [a for a in range(len(all_dates))]

#Добавляем в особую табличку тексты. Для этого подаём функции DataFrame словарь
corpus = pd.DataFrame({'index': index, 'date': all_dates,
                       'title': all_titles, 'text': all_texts,  
                       'link': all_links})

corpus.head(3)

,index,date,title,text,link
0,0,2023/3/24/,Прилетевший в Петербург россиянин напал на там...,"В петербургском аэропорту Пулково россиянин, к...",https://www.mk.ru/incident/2023/03/24/priletev...
1,1,2023/3/24/,Росгвардия нашла 200 книг с экстремистским сод...,"Пресс-служба Росгвардии сообщила, что сотрудни...",https://www.mk.ru/politics/2023/03/24/rosgvard...
2,2,2023/3/24/,Times сообщила о патрулировании у «Северных по...,ВМС Дании и Швеции патрулировали район у «Севе...,https://www.mk.ru/politics/2023/03/24/times-so...
